In [1]:
# This notebook loads tracks from a stack, finds where the tracks cross the rows,
# crops out images before and after the crossing. It saves the images in a girder
# lightbox in the series folder.

from __future__ import division, print_function
from ipywidgets import interact, interactive, fixed, interact_manual    
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy as sp
from scipy import ndimage
from IPython.core.debugger import set_trace
import cv2
import sys
from pprint import pprint
from random import randint
from nuclei.foci import *
import nuclei.girder as g
import nuclei.girder.time_lapse_data as reader

plt.rcParams['image.cmap'] = 'gist_earth'
matplotlib.rcParams['figure.figsize'] = (11.0, 11.0)


# input parameters
czi_folder_id = "5ae21cc31fbb9052b6669e58"

# Setup the reader.
source = reader.time_lapse_data()
source.load(czi_folder_id) 


In [2]:
# This block sets up the reader of a czi folder.
# it then detects rows in the brightfield channel of an image.

def count_foci_image(im):
    # get the foci channel
    red = im[...,2]
    img = red
    
    # band pass
    center_std = 1.5
    # change the surround standard deviation to get rid of spots
    surround_std = 6
    # kernel size has to be an integer and odd. 
    # If it is too small, the tails of gaussians are truncated.
    # if it is too big, the gaussian blur takes a long time to run.
    k = int(center_std*2)*2+1
    center = cv2.GaussianBlur(img,(k,k),center_std)
    k = int(surround_std*2)*2+1
    surround = cv2.GaussianBlur(blue,(k,k),surround_std)
    # Assume Gaussian blur preserves scale (kernel sums to 1)
    # subtrace surround from center to get center surround filter (band pass)
    band_pass = center.astype(np.float)-surround.astype(np.float)    
    img = band_pass

    # threshold, and morphology
    #change this to isolate the spots after filtering
    threshold = 50.0 # 140.0
    ret,out = cv2.threshold(img,threshold,255,cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    mask = cv2.morphologyEx(out, cv2.MORPH_OPEN, kernel)
    img = mask
    
    # count the spots
    img = np.array(img, dtype=np.uint8)
    im2, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    num_features = len(contours)
    return(num_features)
    
def count_foci_item(cross_item):
    images = g.read_item_images(cross_item['_id'])
    description = str(counts)
    print('      ' + description)
    g.set_item_description(cross_item['_id'], description)


def process_crossing(points, point_idx, row_idx, series_idx, source):
    cross_time = points[point_idx][2]
    print("    process crossing: row = %d, time = %d"%(row_idx, cross_time))
    series_folder = source.get_series_folder(series_idx)
    # Assemble a list of images cropped around the cell.
    images = []
    start_idx = max(point_idx - 10, 0);
    stop_idx = min(point_idx + 10, len(points));
    for point_idx in range(start_idx, stop_idx):
        x, y, time = points[point_idx]
        im, item_obj = source.get_image(series_idx, time)
        im = im[int(y)-64:int(y)+64, int(x)-64:int(x)+64, ...]
        if im.shape[0] > 64 and im.shape[1] > 64:
            images.append(im)
    cross_item = g.upload_images(images, 'cross_%d_%d_%d'%(cross_time, row_idx, x), series_folder['_id']) 
    print(cross_item)
    count_foci_item(cross_item)
    
def process_track(track, rows, series_idx, source):
    print("  process track %s"%track['lineColor'])
    # sort the track points by time. The order might have been mixed by the proofreading.
    points = track['points']
    points.sort(key=lambda pt: pt[2])
    if len(points) == 0:
        return
    for row_idx, row in enumerate(rows):
        # calulate the when the track crosses the row of circular barriers.
        pt = points[0]
        above0 = is_above_line(int(row[0]),int(row[1]),pt[0],pt[1]+30)
        for point_idx in range(1,len(points)):
            pt = points[point_idx]
            above1 = is_above_line(int(row[0]),int(row[1]),pt[0],pt[1]+30)
            if above0 and not above1 or above1 and not above0:
                process_crossing(points, point_idx, row_idx, series_idx, source)

def process_series(source, series_idx):
    print("Processing series %d"%series_idx)
    num_time_steps = source.get_series_length(series_idx)
    im, item_obj = source.get_image(series_idx,0)
    # detect rows of circles.
    rows = compute_rows(im)
    if rows == None or len(rows) != 3:
        print("  Could not find rows")
        return
    
    # draw the row as a santity check
    #for row in rows:
    #    cv2.line(im,(10,int(10*row[0]+row[1])),(2000,int(2000*row[0]+row[1])),(255,0,0),5)
    #plt.imshow(im)

    # Load the tracks
    # First get the stack item from the source.
    stack = source.get_series_stack(series_idx)
    # Get the tracks from the stack item
    annotation = g.Annotation("tracks")
    annotation.LoadFromItem(stack['_id'])
    #print(annotation)
    tracks = annotation.annot_obj['elements']
    
    for track in tracks:
        process_track(track, rows, series_idx, source)
        

In [3]:

num_series = source.get_number_of_series()
#for idx in range(num_series):
for idx in range(1):
    process_series(source, idx)
        
        
        

Processing series 0
annotation_id = 5ae279861fbb9052b666c107
  process track #8dd663
  process track #c84a7f
  process track #e1b005
  process track #ecf61f
  process track #432d03
  process track #97a286
  process track #82320c
  process track #e22b8b
  process track #dbd6da
  process track #15687e
  process track #c5dc13
  process track #96370a
  process track #f0f6e8
  process track #081551
  process track #43de63
  process track #619baa
  process track #678d70
  process track #89ca1f
    process crossing: row = 2, time = 49
None


TypeError: 'NoneType' object is not subscriptable